In [2]:
import pandas as pd
import sys, os
import logging, json
import pandas as pd
from tqdm.notebook import tqdm
from influxdb import InfluxDBClient

In [3]:
client_influx = InfluxDBClient(host='127.0.0.1', port=8086, username='admin', password='password', database='btc')
def read_influx(query, client=client_influx):
    logging.info(query)
    return pd.DataFrame(client.query(query, chunked=True, chunk_size=10000).get_points())
read_influx("SHOW MEASUREMENTS")

,name
0,book
1,ticker
2,trades


In [63]:
iql ="""SELECT * FROM "ticker" WHERE ("feed" = 'BINANCE_FUTURES' AND "pair" = 'BTC-USDT') AND time>now()-1h"""
df_quote = read_influx(iql)

In [64]:
iql ="""SELECT * FROM "trades" WHERE ("feed" = 'BINANCE_FUTURES' AND "pair" = 'BTC-USDT') AND time>now()-1h"""
df_trade = read_influx(iql)

In [69]:
#df_quote.to_csv('quote.csv')
#df_trade.to_csv('trade.csv')

In [84]:
df_quote['time'] = pd.to_datetime(df_quote['time'])
df_trade['time'] = pd.to_datetime(df_trade['time'])

In [86]:
df_taq = df_trade[['amount', 'price', 'side']].merge(df_quote[['ask', 'ask_size', 'bid', 'bid_size']], on='time', how='outer')

In [90]:
df_taq.sort_index().to_csv('taq.csv')

In [4]:
pd.read_csv('taq.csv').head()

,time,amount,price,side,ask,ask_size,bid,bid_size
0,2021-04-19 05:58:25.757999872+00:00,0.054,56565.57,sell,NaN,NaN,NaN,NaN
1,2021-04-19 05:58:25.937999872+00:00,NaN,NaN,NaN,56565.58,1.745,56565.57,0.521
2,2021-04-19 05:58:25.969000192+00:00,NaN,NaN,NaN,56565.58,1.745,56565.57,0.515
3,2021-04-19 05:58:25.974999808+00:00,NaN,NaN,NaN,56565.58,1.745,56565.57,0.520
4,2021-04-19 05:58:26.006000128+00:00,NaN,NaN,NaN,56565.58,1.465,56565.57,0.520
